In [1]:
import keras

Using TensorFlow backend.
/home/kaiser17/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
reuse=tf.AUTO_REUSE

In [3]:
from keras.datasets import mnist


In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [5]:
print(x_train.shape)

(60000, 28, 28)


In [6]:
x_train=x_train.reshape(60000,28*28)
y_train=y_train.reshape(60000,)
x_train2 = x_train/255.
x_test2 = x_test/255.

In [7]:
x_train2=np.transpose(x_train2)

print(x_train2.shape)
print(y_train.shape)


(784, 60000)
(60000,)


In [8]:
y_trainV2=tf.one_hot(y_train,10)

In [9]:
ss=tf.Session()
y_trainV2=ss.run(y_trainV2)
y_trainV2=np.transpose(np.array(y_trainV2))
y_trainV2.shape

(10, 60000)

In [10]:
z=x_train2.reshape(60000,28,28)

In [11]:
def sigmoid(z):
    x=tf.placeholder(tf.float32,name="x")
    ss=tf.sigmoid(x)
    j=tf.Session()
    l=j.run(ss,feed_dict={x:z})
    return l

In [12]:
def computeCost(Z3,Y):
    q=tf.transpose(Z3)
    r=tf.transpose(Y)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=q,labels=r))
    return cost


In [13]:
def Initialize():    
    tf.set_random_seed(1)                   
    with tf.variable_scope(tf.get_variable_scope(),reuse=tf.AUTO_REUSE) as scope:
        
        W1 = tf.get_variable("W1", [300,784], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
        b1 = tf.get_variable("b1", [300,1], initializer = tf.zeros_initializer())
        W2 = tf.get_variable("W2", [100,300], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
        b2 = tf.get_variable("b2", [100,1], initializer = tf.zeros_initializer())
        W3 = tf.get_variable("W3", [10,100], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
        b3 = tf.get_variable("b3", [10,1], initializer = tf.zeros_initializer())
    

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2,
                  "W3": W3,
                  "b3": b3}
    
    return parameters
   

In [14]:
def forwardProp(X,dictw):
    W1=dictw['W1']
    W2=dictw['W2']
    W3=dictw['W3']
    b1=dictw['b1']
    b2=dictw['b2']
    b3=dictw['b3']
    Z1 = tf.add(tf.matmul(W1,X),b1)                                              # Z1 = np.dot(W1, X) + b1
    A1 = tf.nn.relu(Z1)                                           # A1 = relu(Z1)
    Z2 =  tf.add(tf.matmul(W2,A1),b2)                                              # Z2 = np.dot(W2, a1) + b2
    A2 = tf.nn.relu(Z2)                                          # A2 = relu(Z2)
    Z3 = tf.add(tf.matmul(W3,A2),b3)
    return(Z3)
    

In [15]:
def create_placeholders(n_x, n_y):
    X= tf.placeholder(tf.float32, [n_x, None], name="X")
    Y = tf.placeholder(tf.float32, [n_y, None], name="Y")
    return X, Y

In [28]:
def model(X_train, Y_train, X_test, Y_test, learning_rate = 0.001,num_epochs = 10, minibatch_size = 30):
    dictw=Initialize()
    (n_x,m)=X_train.shape
   
    n_y=Y_train.shape[0]
    print(n_x)
    X,Y=create_placeholders(n_x,n_y)
    Z3=forwardProp(X,dictw)
    cost=computeCost(Z3,Y)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(cost)
    init = tf.global_variables_initializer()
    with tf.Session() as ss: 
        ss.run(init)
        for i in range(num_epochs):
            l=m//minibatch_size
            #print(num_epochs)
            r=0
            epoch_cost=0
            for g in range(l):
                x1=X_train[:,r:r+minibatch_size]
                y1=Y_train[:,r:r+minibatch_size]
                r=r+minibatch_size
                #print(x1.shape)
                _ , minibatch_cost = ss.run([optimizer, cost], feed_dict={X:x1, Y:y1})
                #epoch_cost += minibatch_cost / l
            #print(epoch_cost)
    # lets save the parameters in a variable
    #parameters = ss.run(dictw)
        print ("Parameters have been trained!")
        
        
    
        parameters = ss.run(dictw)
        print ("Parameters have been trained!")

        # Calculate the correct predictions
        correct_prediction = tf.equal(tf.argmax(Z3), tf.argmax(Y))

        
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

        print ("Train Accuracy:", accuracy.eval({X: X_train, Y: Y_train}))
    return(parameters)
    

In [29]:

pm=model(X_train=x_train2,Y_train=y_trainV2,X_test=x_test2,Y_test=y_test)

784
Parameters have been trained!
Parameters have been trained!
Train Accuracy: 0.914667


In [35]:
testzz=x_test2.reshape(10000,784)
Xtestzz=np.transpose(testzz)
y_TEst=tf.one_hot(y_test,10)
ss=tf.Session()
y_testzz=ss.run(y_TEst)
y_testzz=np.transpose(np.array(y_testzz))
ss.close()

In [36]:
print(Xtestzz.shape)
print(y_testzz.shape)

(784, 10000)
(10, 10000)


In [40]:
(n_xz,m)=Xtestzz.shape
   
n_yz=y_testzz.shape[0]

X,Y=create_placeholders(n_xz,n_yz)
ss=tf.Session()
q=forwardProp(X,pm)
ans=q


In [41]:
correct_prediction = tf.equal(tf.argmax(ans), tf.argmax(y_testzz))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [42]:
print(ss.run(accuracy,feed_dict={X:Xtestzz,Y:y_testzz}))

0.9211
